In [1]:
!pip install transformers==4.44.2

In [2]:
import re
import torch
from transformers import BertTokenizer, BertModel
from datasets import load_dataset
import pandas as pd

In [3]:
datos=pd.read_csv("dataset_dialogos/dataset_dialogos.csv")

In [4]:
datos

,source_file,dialogue
0,130000.jsonl,Está saliendo la luna.\nNo podemos perder tiempo.
1,130000.jsonl,Está descansando.\nEsperando una nueva vida.
2,130000.jsonl,No.\nVamos. No puede hacerte nada.
3,130000.jsonl,Tiene el cuello roto.\nEl cerebro no sirve.
4,130000.jsonl,"Gracias, caballeros.\nLa clase ha terminado."
...,...,...
19983467,89999.jsonl,¿De veras?\nSí.
19983468,89999.jsonl,"Ah, sí.\nSí."
19983469,89999.jsonl,¿Entonces me amas un poco?\nNo.
19983470,89999.jsonl,"Si te beso, ¿me dispararân?\nNo."


In [5]:
datos_preprocesado=datos[:50]["dialogue"]
datos_preprocesado

0     Está saliendo la luna.\nNo podemos perder tiempo.
1          Está descansando.\nEsperando una nueva vida.
2                    No.\nVamos. No puede hacerte nada.
3           Tiene el cuello roto.\nEl cerebro no sirve.
4          Gracias, caballeros.\nLa clase ha terminado.
5            Me alegro mucho de verte.\n¿Qué te ocurre?
6     Has tenido noticias de Henry.\nSí. Ya hacía cu...
7     Tengo miedo.\nLa he leído una y otra vez, pero...
8     "Vivo en una atalaya abandonada cerca de la vi...
9     Sus experimentos.\nSí, eso es lo que me asusta...
10                                  Victor.\nLo siento.
11    Buenas noches, Victor. Y gracias.\nBuenas noch...
12                                       Victor.\n¿Qué?
13    Voy contigo.\n¡Ni hablar!\nTengo que hacerlo. ...
14    Herr Frankenstein es un joven tan brillante co...
15    Sus estudios del galvanismo químico y la elect...
16    Herr Frankenstein ha cambiado mucho.\n¿Ha camb...
17    Encontró lo que buscaba en otra parte.\nCa

In [6]:
#Prueba del tokenizer y embeddings model

In [7]:
from transformers import BertTokenizer,BertModel
model_name="bert-base-multilingual-cased"
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)
texto = """ "a ¿qué hora "pasa el, ciento siete.   """
tokens = tokenizer.tokenize(texto)
print(tokens)
# ['a', 'qué', 'hora', 'pasa', 'el', 'cien', '##to', 'siete']
tokens_id=tokenizer.convert_tokens_to_ids(tokens)
#[169, 38188, 24301, 26088, 10125, 99485, 10340, 28394]
embedding_vector = model.embeddings.word_embeddings.weight[tokens_id[0]]
embedding_vector.shape

['"', 'a', '¿', 'qué', 'hora', '"', 'pasa', 'el', ',', 'cien', '##to', 'siete', '.']


C:\Users\emili\miniconda3\envs\labo\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


torch.Size([768])

In [29]:
#Funciones de etiquetado
from typing import List, Union
from transformers import BertTokenizer,BertModel

model_name="bert-base-multilingual-cased"
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

def separar_palabras(texto:str)->List[str]:
    primera_letra=0
    palabras=[]
    
    #palabras que terminan por espacio
    for i in range(len(texto)):
        if texto[i] == " ":
            palabra=texto[primera_letra:i]
            primera_letra=i+1
            palabras.append(palabra)
        #palabras que terminan por cambio de parrafo
        if texto[i]=="\n":
            palabra=texto[primera_letra:i]
            primera_letra=i+1  
            palabras.append(palabra)

        if texto[i]=="¡" or texto[i]=='"':
            primera_letra+=1
    
    #agrego ultima palabra
    palabras.append(texto[primera_letra:])
    return palabras

def clasificar_mayusculas(palabra:str)->int:
    """Toma una palabra y cuenta sus mayusculas"""
    cantidad=0
    for i in range(len(palabra)):
        if palabra[i].lower()!=palabra[i]:
            cantidad+=1
    res=0
    if cantidad==len(palabra): #todas son mayuscula
        res=3
    elif cantidad==1 and palabra[0].lower()!=palabra[0]: #solo la primera es mayuscula
        res=1
    elif cantidad>0: #alguna es mayuscula (iPhone tiene que estar aca)
        res=2
    else: #Ninguna de las anteriores
        res=0
    return res
    
def asignar_labels(datos: pd.Series, path=None)->pd.DataFrame:
    punt_inicial=[]
    punt_final=[]
    instancias=[]
    token_ids=[]
    tokens=[]
    capitalizacion=[]
    instancia=0
    print("Comenzando procesamiento")
    for dialogo in datos:
        if instancia % (len(datos)//10)==0:
            print(f"procesando un {instancia/len(datos)*100}%")
            #print(len(capitalizacion))
            #print(len(token_ids))
            #print(len(tokens))
            #print(len(instancias))
            #print(len(punt_inicial))
            #print(len(punt_final))
        
        
        palabras_originales= separar_palabras(dialogo)
        palabras_lower= [palabra.lower() for palabra in palabras_originales]
        instancia+=1
        #print(palabras_lower)

        punt_inicial_seguridad=[]
        punt_final_seguridad=[]
        instancias_seguridad=[]
        capitalizacion_seguridad=[]
        palabra_tokenizada_ids_seguridad=[]
        tokens_seguridad=[]
        es_seguro=True
        
        for idx, palabra in enumerate(palabras_lower):
            
            hay_signo_apertura=False
            palabra_tokenizada= tokenizer.tokenize(palabra)

            #Recordartorio SON PALABRAS. ejemplos: '¿Que' o "escuela?" o "Franquito," o "Emi."  "¿ que ?"
            if len(palabra_tokenizada)>0:
                if palabra_tokenizada[0]=="¿":
                    palabra_tokenizada=palabra_tokenizada[1:]
                    hay_signo_apertura=True
                    punt_inicial_seguridad.append("¿")
                

            if len(palabra_tokenizada)>0:
                if palabra_tokenizada[-1]=="?":
                    palabra_tokenizada=palabra_tokenizada[:-1]
                    punt_final_seguridad+=[None]*(len(palabra_tokenizada)-1)
                    punt_final_seguridad.append("?")
                    
                
        
                elif palabra_tokenizada[-1]==".":
                    palabra_tokenizada=palabra_tokenizada[:-1]
        
                    punt_final_seguridad+=[None]*(len(palabra_tokenizada)-1)
                    punt_final_seguridad.append(".")
        
                elif palabra_tokenizada[-1]==",":
                    palabra_tokenizada=palabra_tokenizada[:-1]
                    punt_final_seguridad+=[None]*(len(palabra_tokenizada)-1)
                    punt_final_seguridad.append(",")
        
                else:
                    punt_final_seguridad+=[None]*(len(palabra_tokenizada))

            
            palabra_tokenizada_ids= tokenizer.convert_tokens_to_ids(palabra_tokenizada) 
            if hay_signo_apertura:
                punt_inicial_seguridad+=[None]*(len(palabra_tokenizada)-1)
                
            else:
                punt_inicial_seguridad+=[None]*(len(palabra_tokenizada))
            #capitalizacion
            clasificacion=None
            palabra_orginal= palabras_originales[idx]
            if len(palabra_tokenizada)>0:
                if palabra_orginal[0]=="¿" or palabra_orginal[0]=='"' or palabra_orginal[0]=='¡':
                    clasificacion= clasificar_mayusculas(palabra_orginal[1:])
                else:   
                    clasificacion= clasificar_mayusculas(palabra_orginal)

            if len(palabra_tokenizada)==0:
                es_seguro=False 
                break
            
            capitalizacion_seguridad+= [clasificacion]*len(palabra_tokenizada)
            palabra_tokenizada_ids_seguridad+=palabra_tokenizada_ids
            tokens_seguridad+=palabra_tokenizada
            instancias_seguridad+=[instancia]*len(palabra_tokenizada)
        
        if es_seguro:
            capitalizacion+=capitalizacion_seguridad
            token_ids+= palabra_tokenizada_ids_seguridad
            tokens+= tokens_seguridad
            instancias+=instancias_seguridad
            punt_inicial+=punt_inicial_seguridad
            punt_final+=punt_final_seguridad

            if len(capitalizacion)!=len(punt_inicial):
                print("dif inicial",instancia,dialogo)
                return
            if len(capitalizacion)!=len(punt_final):
                print("dif final",instancia,dialogo)
                return
                
        else:
            instancia-=1
    
    
        if len(capitalizacion)!=len(punt_inicial):
            print("dif inicial",instancia)
        if len(capitalizacion)!=len(punt_final):
            print("dif final",instancia,)
    
    datos_listos= pd.DataFrame({"instancia_id":instancias,"token":tokens,"token_id":token_ids,"punt_inicial":punt_inicial,"punt_final":punt_final,"capitalizacion":capitalizacion})    

    if path!=None:
        datos_listos.to_csv(path,index=False)
    return datos_listos    
            

datos_listos=asignar_labels(datos_preprocesado,"datos_modelo.csv")

C:\Users\emili\miniconda3\envs\labo\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Comenzando procesamiento
procesando un 0.0%
procesando un 10.0%
procesando un 20.0%
procesando un 30.0%
procesando un 40.0%
procesando un 50.0%
procesando un 60.0%
procesando un 70.0%
procesando un 80.0%
procesando un 90.0%


In [30]:
separar_palabras(datos["dialogue"][0]), datos["dialogue"][0]

(['Está', 'saliendo', 'la', 'luna.', 'No', 'podemos', 'perder', 'tiempo.'],
 'Está saliendo la luna.\nNo podemos perder tiempo.')

In [9]:
datos_listos[(datos_listos["instancia_id"]==48)& (datos_listos["capitalizacion"]==3)], datos_preprocesado[47]

(      instancia_id token  token_id punt_inicial punt_final  capitalizacion
 1838            48     y       193         None       None               3,
 'Qué más da. Al fin y al cabo, es sólo tejido muerto.\nSólo podría engendrar el mal.\nSu salud se deteriorará si continúa con esta locura.\nEstoy asombrosamente cuerdo, doctor.\nHa creado un monstruo y éste le destruirá.\nPaciencia.\nCreo en este monstruo, como usted lo llama.\nY si usted no cree en él, será mejor que se marche.\nPiense en Elizabeth. En su padre.\nElizabeth cree en mí.\n¿Mi padre? Jamás cree en nadie.')

In [ ]:
datos_primer_decima_parte=datos["dialogue"].loc[:len(datos)//10]

datos_listos=asignar_labels(datos_primer_decima_parte,"datos_modelo.csv")

Comenzando procesamiento
procesando un 0.0%
procesando un 9.999959966932687%
procesando un 19.999919933865375%
procesando un 29.99987990079806%
procesando un 39.99983986773075%


In [32]:
datos_listos

,instancia_id,token,token_id,punt_inicial,punt_final,capitalizacion
0,1,está,11559,None,None,1
1,1,sal,31119,None,None,0
2,1,##iendo,18426,None,None,0
3,1,la,10109,None,None,0
4,1,luna,44885,None,.,0
...,...,...,...,...,...,...
725015,19572,chocolate,87212,None,None,0
725016,19572,cal,25923,None,None,0
725017,19572,##iente,22847,None,.,0
725018,19572,sí,18419,None,None,1


In [11]:
separar_palabras(datos["dialogue"][128])

['Pareces', 'un', 'poco', 'nervioso.', 'Yo,', '¿nervioso.', '?', 'No,', 'yo.']

In [ ]:
def asignar_labels_fraccional(datos: pd.Series, path: str =None,verbose: bool=False,prop: float =0.1)->pd.DataFrame:
   
    instancia=0
    print("Comenzando procesamiento")
    max_intervalos=int(1/prop):
    for intervalos in max_intervalos:
        datos_int=datos[intervalos*len(datos)*prop,(intervalos+1)*len(datos)*prop]
        datos_listos=pd.Dataframe({})
        punt_inicial=[]
        punt_final=[]
        instancias=[]
        token_ids=[]
        tokens=[]
        capitalizacion=[]
        for dialogo in datos:
        if instancia % (len(datos)//10)==0:
            print(f"procesando un {instancia/len(datos)*100}%")
            #print(len(capitalizacion))
            #print(len(token_ids))
            #print(len(tokens))
            #print(len(instancias))
            #print(len(punt_inicial))
            #print(len(punt_final))
        
        
        palabras_originales= separar_palabras(dialogo)
        palabras_lower= [palabra.lower() for palabra in palabras_originales]
        instancia+=1
        #print(palabras_lower)

        punt_inicial_seguridad=[]
        punt_final_seguridad=[]
        instancias_seguridad=[]
        capitalizacion_seguridad=[]
        palabra_tokenizada_ids_seguridad=[]
        tokens_seguridad=[]
        es_seguro=True
        
        for idx, palabra in enumerate(palabras_lower):
            
            hay_signo_apertura=False
            palabra_tokenizada= tokenizer.tokenize(palabra)

            #Recordartorio SON PALABRAS. ejemplos: '¿Que' o "escuela?" o "Franquito," o "Emi."  "¿ que ?"
            if len(palabra_tokenizada)>0:
                if palabra_tokenizada[0]=="¿":
                    palabra_tokenizada=palabra_tokenizada[1:]
                    hay_signo_apertura=True
                    punt_inicial_seguridad.append("¿")
                

            if len(palabra_tokenizada)>0:
                if palabra_tokenizada[-1]=="?":
                    palabra_tokenizada=palabra_tokenizada[:-1]
                    punt_final_seguridad+=[None]*(len(palabra_tokenizada)-1)
                    punt_final_seguridad.append("?")
                    
                
        
                elif palabra_tokenizada[-1]==".":
                    palabra_tokenizada=palabra_tokenizada[:-1]
        
                    punt_final_seguridad+=[None]*(len(palabra_tokenizada)-1)
                    punt_final_seguridad.append(".")
        
                elif palabra_tokenizada[-1]==",":
                    palabra_tokenizada=palabra_tokenizada[:-1]
                    punt_final_seguridad+=[None]*(len(palabra_tokenizada)-1)
                    punt_final_seguridad.append(",")
        
                else:
                    punt_final_seguridad+=[None]*(len(palabra_tokenizada))

            
            palabra_tokenizada_ids= tokenizer.convert_tokens_to_ids(palabra_tokenizada) 
            if hay_signo_apertura:
                punt_inicial_seguridad+=[None]*(len(palabra_tokenizada)-1)
                
            else:
                punt_inicial_seguridad+=[None]*(len(palabra_tokenizada))
            #capitalizacion
            clasificacion=None
            palabra_orginal= palabras_originales[idx]
            if len(palabra_tokenizada)>0:
                if palabra_orginal[0]=="¿" or palabra_orginal[0]=='"' or palabra_orginal[0]=='¡':
                    clasificacion= clasificar_mayusculas(palabra_orginal[1:])
                else:   
                    clasificacion= clasificar_mayusculas(palabra_orginal)

            if len(palabra_tokenizada)==0:
                es_seguro=False 
                break
            
            capitalizacion_seguridad+= [clasificacion]*len(palabra_tokenizada)
            palabra_tokenizada_ids_seguridad+=palabra_tokenizada_ids
            tokens_seguridad+=palabra_tokenizada
            instancias_seguridad+=[instancia]*len(palabra_tokenizada)
        
        if es_seguro:
            capitalizacion+=capitalizacion_seguridad
            token_ids+= palabra_tokenizada_ids_seguridad
            tokens+= tokens_seguridad
            instancias+=instancias_seguridad
            punt_inicial+=punt_inicial_seguridad
            punt_final+=punt_final_seguridad

            if len(capitalizacion)!=len(punt_inicial):
                print("dif inicial",instancia,dialogo)
                return
            if len(capitalizacion)!=len(punt_final):
                print("dif final",instancia,dialogo)
                return
                
        else:
            instancia-=1
    
    
        if len(capitalizacion)!=len(punt_inicial):
            print("dif inicial",instancia)
        if len(capitalizacion)!=len(punt_final):
            print("dif final",instancia,)
    
    datos_listos= pd.DataFrame({"instancia_id":instancias,"token":tokens,"token_id":token_ids,"punt_inicial":punt_inicial,"punt_final":punt_final,"capitalizacion":capitalizacion})    

    if path!=None:
        datos_listos.to_csv(path,index=False)
    return datos_listos    
    

In [ ]:
datos

In [ ]:
from datasets import load_datas/

num_muestras = 1000

print(f"Cargando {num_muestras} muestras en 'modo streaming'")

try:
    dataset_iterable = load_dataset(
        path="openbmb/DCAD-2000",
        name="spa_Latn",
        split="train",
        streaming=True
    )

    print("\nDataset listo en modo streaming.")
    muestras_tomadas = dataset_iterable.take(num_muestras)
    lista_de_muestras = list(muestras_tomadas)

except Exception as e:
    print(f"\nError al cargar el dataset en modo streaming: {e}")

### Visualizar de Muestras

In [ ]:

print(f"--- Visualizando las {len(lista_de_muestras)} muestras descargadas ---")

for i, muestra in enumerate(lista_de_muestras):

    #print(muestra)

    if 'text' in muestra:
        a = 2
        print(f"\n[Muestra {i + 1}]")
        print(muestra['text'])
    else:
        print(f"\n[Muestra {i + 1}] No se encontró la clave 'text'.")
        print("Contenido de la muestra:")
        print(muestra)

print("\n--- Fin de la visualización ---")

In [ ]:
def conseguir_palabras(texto, puntuacion_a_ignorar='()¡!:;…"'):
    # quitar la puntuacion que no nos interesa predecir.
    # Podria haber razones para decidir reemplazar un ... por . o hacer que los signos de exclamacion pongan mayusculas
    # A priori, no invente reglas asi.

    # Quizas si venga bien poner una regla que diga reemplazar '(' y ')' por ', '
    translator = str.maketrans('', '', puntuacion_a_ignorar)
    texto_con_puntuacion_a_adivinar = texto.translate(translator)
    
    # Me gusto la idea de usar regex
    texto_con_puntuacion_a_adivinar = re.sub(r'\.+', '.', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r'\?+', '?', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r',+', ',', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r'¿+', '¿', texto_con_puntuacion_a_adivinar)

    # Evitar que hola . se separe en palabras 'hola' y '.'. Lo mismo para el resto de las puntuaciones
    texto_con_puntuacion_a_adivinar = re.sub(r' +\.', '.', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r' +,', ',', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r' +\?', '?', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r'¿ +', '¿', texto_con_puntuacion_a_adivinar)

    # Sacar saltos de linea y otros tipos de cosas (Desconozco la lista completa)
    texto_monoespaciado = re.sub(r'\s+', ' ', texto_con_puntuacion_a_adivinar).strip()
    palabras = texto_monoespaciado.split()
    return palabras

def transformar_input(texto):
    palabras_y_puntuaciones = conseguir_palabras(texto)
    palabras = []

    translator = str.maketrans('', '', '¿?,.')

    for palabra in palabras_y_puntuaciones:
        abre_pregunta = '¿' in palabra
        cierra_pregunta = '?' in palabra
        continua_texto = ',' in palabra
        cierra_texto = '.' in palabra
        valor_puntuaciones = [abre_pregunta, cierra_pregunta, continua_texto, cierra_texto]

        palabra_sin_puntuacion = palabra.translate(translator)
        palabras.append([palabra_sin_puntuacion, valor_puntuaciones])
    
    return palabras

print(lista_de_muestras[0].get('text'))
print()
asd = transformar_input(lista_de_muestras[0].get('text'))
asd

# Permitimos que haya mas de una puntuacion al inicio o al final?
#asd = conseguir_palabras("¿     Hola,    ?.")
#asd == ['¿Hola,?.']

# Que hacemos con estos casos?
#dsa = conseguir_palabras("asd.asd")
#dsa == ['asd.asd']

In [ ]:
import re
import torch
from transformers import BertTokenizer, BertModel

textos_originales= []
textos_aplanados = []
PUNT_TODOS = '¿¡.,?!:;…"'

for i, muestra in enumerate(lista_de_muestras):

    # Obtener texto_original (usando la clave 'text')
    texto_original = muestra.get('text')
    textos_originales.append(texto_original)
    if not texto_original:
        print(f"Saltando muestra {i}, clave 'text' no encontrada o vacía.")
        continue

    # Crear texto_plano (todo minúsculas, sin puntuación)
    texto_lower = texto_original.lower()
    # quitar toda la puntuación
    translator = str.maketrans('', '', PUNT_TODOS)
    texto_plano = texto_lower.translate(translator)
    # Reemplazar múltiples espacios por uno
    texto_plano = re.sub(r'\s+', ' ', texto_plano).strip()
    textos_aplanados.append(texto_plano)


In [ ]:
textos_originales[0]

In [ ]:
textos_aplanados[0]